In [1]:
import numpy as np 
import pandas as pd 
import os 

In [34]:
# for dir_entry in os.scandir("./"):
#     if not dir_entry.name.startswith('.'):
#         print(dir_entry.name)

In [2]:
os.listdir()

['106_2020-01-01T0000_2024-05-17T0000.csv',
 '106_2021-01-01T0000_2024-06-04T0000.csv',
 '1_2020-01-01T0000_2024-05-17T0000.csv',
 '22_2021-01-01T0000_2024-06-04T0000.csv',
 '244_2020-01-01T0000_2024-05-17T0000.csv',
 '244_2021-01-01T0000_2024-06-04T0000.csv',
 '251_2021-01-01T0000_2024-12-31T0000.csv',
 '287_2020-01-01T0000_2024-05-17T0000.csv',
 '288_2020-01-01T0000_2024-05-17T0000.csv',
 '319_2021-01-01T0000_2024-06-04T0000.csv',
 '51_2021-01-01T0000_2024-12-31T0000.csv',
 '52_2021-01-01T0000_2024-12-31T0000.csv',
 '93_2021-01-01T0000_2024-06-04T0000.csv',
 'data_2020-01-01T0000_2024-05-17T0000.csv',
 'data_preparing.ipynb',
 'FI',
 'fi_ee.csv',
 'fi_no4.csv',
 'fi_se1.csv',
 'fi_se1_01_08_21.csv',
 'fi_se3.csv',
 'idf',
 'node_modules',
 'Oomi Spot-hintatieto (1_06_2023-30_09_2023).csv',
 'Oomi Spot-hintatieto (1_10_2023-31_10_23).csv',
 'Oomi Spot-hintatieto (1_11_2023-31_12_2023).csv',
 'Oomi Spot-hintatieto (1_1_2023-31_05_2023).csv',
 'Oomi Spot-hintatieto (1_1_2024-31_03_2024)

In [49]:
# os.listdir("ERCOT")

In [50]:
# os.listdir("ERCOT/SPP/2020")

In [51]:
# os.listdir("ERCOT/CCP/2020")

In [52]:
# spp_jan = pd.read_excel('ERCOT/SPP/2020/rpt.00013060.0000000000000000.DAMLZHBSPP_2020.xlsx', )

In [53]:
# spp_jan

In [54]:
# ccp = pd.read_csv('ERCOT/CCP/2020/rpt.00013091.0000000000000000.20210101080006.DAMASMCPC_2020.csv')

In [36]:
# ccp

In [99]:
# Define the starting prefixes of the new data
prefixes = ["51", "52", "251", "106", "319", "22", "93", "244"]
# Load the CSV files into New_data
New_data = [pd.read_csv(f"{f}") for f in os.listdir() if f.endswith(".csv") and any(f.startswith(prefix) for prefix in prefixes)]


In [100]:
[f for f in os.listdir() if f.endswith(".csv") and any(f.startswith(prefix) for prefix in prefixes)]

['106_2020-01-01T0000_2024-05-17T0000.csv',
 '106_2021-01-01T0000_2024-06-04T0000.csv',
 '22_2021-01-01T0000_2024-06-04T0000.csv',
 '244_2020-01-01T0000_2024-05-17T0000.csv',
 '244_2021-01-01T0000_2024-06-04T0000.csv',
 '251_2021-01-01T0000_2024-12-31T0000.csv',
 '319_2021-01-01T0000_2024-06-04T0000.csv',
 '51_2021-01-01T0000_2024-12-31T0000.csv',
 '52_2021-01-01T0000_2024-12-31T0000.csv',
 '93_2021-01-01T0000_2024-06-04T0000.csv']

In [71]:
[x.shape for x in New_data]

[(38351, 3),
 (30109, 3),
 (2452, 3),
 (38351, 3),
 (29898, 3),
 (677, 3),
 (23639, 3),
 (29833, 3),
 (29833, 3),
 (7321, 3)]

In [96]:
# Preprocess and reorder columns
for i, df in enumerate(New_data):
    if 'Start time UTC' in df.columns and 'End time UTC' in df.columns:
        df.rename(columns={'Start time UTC': 'Start', 'End time UTC': 'End'}, inplace=True)
        df['Start'] = pd.to_datetime(df['Start'])
        df['End'] = pd.to_datetime(df['End'])
        df.drop(columns=["Start"], inplace=True)
        df.sort_values("End", inplace=True)
        df["Delivery Date"] = pd.to_datetime(df["End"], format="mixed").dt.date
        df["Hour Ending"] = pd.to_datetime(df["End"], format="mixed").dt.time.apply(lambda x: x.strftime("%H:%M"))
        df["Settlement Point"] = "SUOMI"
        df["Repeated Hour Flag"] = df.groupby(["Delivery Date", "Hour Ending"])["Settlement Point"].transform("count").apply(lambda x: "Y" if x > 1 else "N")
        df.drop(columns=["End"], inplace=True)        
        # Specify the order of the first four columns
        ordered_columns = ["Delivery Date", "Hour Ending", "Settlement Point", "Repeated Hour Flag"]
        # Append the rest of the columns
        remaining_columns = [col for col in df.columns if col not in ordered_columns]
        df = df[ordered_columns + remaining_columns]        
        # Update the DataFrame in the list with the reordered columns
        New_data[i] = df



In [102]:
# first_csv = New_data[9] if New_data else None
# print(first_csv)

                Start time UTC              End time UTC  \
0     2020-12-31T21:00:00.000Z  2020-12-31T22:00:00.000Z   
1     2020-12-31T22:00:00.000Z  2020-12-31T23:00:00.000Z   
2     2020-12-31T23:00:00.000Z  2021-01-01T00:00:00.000Z   
3     2021-01-01T00:00:00.000Z  2021-01-01T01:00:00.000Z   
4     2021-01-01T01:00:00.000Z  2021-01-01T02:00:00.000Z   
...                        ...                       ...   
7316  2021-11-01T17:00:00.000Z  2021-11-01T18:00:00.000Z   
7317  2021-11-01T18:00:00.000Z  2021-11-01T19:00:00.000Z   
7318  2021-11-01T19:00:00.000Z  2021-11-01T20:00:00.000Z   
7319  2021-11-01T20:00:00.000Z  2021-11-01T21:00:00.000Z   
7320  2021-11-01T21:00:00.000Z  2021-11-01T22:00:00.000Z   

      The sales price of production imbalance electricity  
0                                              26.55000    
1                                              24.07000    
2                                              24.94999    
3                                      

In [49]:
oomis = [pd.read_csv(f"{f}") for f in os.listdir() if f.endswith(".csv") and f.startswith("Oomi")]

In [50]:
# [f for f in os.listdir() if f.endswith(".csv") and f.startswith("Oomi")]

In [51]:
[x.shape for x in oomis]

[(2928, 2), (744, 2), (1464, 2), (3624, 2), (2184, 2), (850, 2), (8734, 2)]

In [52]:
oomis = pd.concat(oomis)

In [53]:
oomis["c/mWh"] = oomis["c/kWh"]*1000

In [54]:
oomis.drop(columns=["c/kWh"], inplace=True)

In [55]:
oomis.sort_values("Time", inplace=True)

In [56]:
oomis["Delivery Date"] = pd.to_datetime(oomis["Time"], format="mixed").dt.date
oomis["Hour Ending"] = pd.to_datetime(oomis["Time"], format="mixed").dt.time.apply(lambda x: x.strftime("%H:%M"))
oomis["Settlement Point"] = "SUOMI"
oomis["Repeated Hour Flag"] = oomis.groupby(["Delivery Date", "Hour Ending"])["Settlement Point"].transform("count").apply(lambda x: "Y" if x > 1 else "N")
oomis["Settlement Point Price"] = oomis["c/mWh"]
oomis.drop(columns=["Time", "c/mWh"], inplace=True)

In [41]:
def get_data_by_year_and_month(df, year, month_name):
    return df.loc[((pd.to_datetime(df["Delivery Date"]).dt.year==year) & (pd.to_datetime(df["Delivery Date"]).dt.month_name().apply(lambda x: x[:3])==month_name))]
get_data_by_year_and_month(oomis, 2024, "Mar").loc[get_data_by_year_and_month(oomis, 2024, "Mar")["Delivery Date"].astype(str)=="2024-03-31"]

,Delivery Date,Hour Ending,Settlement Point,Repeated Hour Flag,Settlement Point Price
2160,2024-03-31,00:00,SUOMI,N,5810.0
2161,2024-03-31,01:00,SUOMI,N,5810.0
2162,2024-03-31,02:00,SUOMI,N,5800.0
2163,2024-03-31,04:00,SUOMI,N,5810.0
2164,2024-03-31,05:00,SUOMI,N,6020.0
2165,2024-03-31,06:00,SUOMI,N,6320.0
2166,2024-03-31,07:00,SUOMI,N,6120.0
2167,2024-03-31,08:00,SUOMI,N,6660.0
2168,2024-03-31,09:00,SUOMI,N,6790.0
2169,2024-03-31,10:00,SUOMI,N,6700.0


In [46]:
oomis.groupby(["Delivery Date", "Hour Ending"])["Settlement Point"].transform("count").apply(lambda x: "Y" if x > 1 else "N")#.value_counts()

0       N
1       N
2       N
3       N
4       N
       ..
2179    N
2180    N
2181    N
2182    N
2183    N
Name: Settlement Point, Length: 20528, dtype: object

In [47]:
oomis["Delivery Date"] = pd.to_datetime(oomis["Delivery Date"], format="%Y-%m-%d").dt.strftime("%m/%d/%Y")

In [48]:
oomis.duplicated().sum()

1

In [49]:
oomis.loc[oomis.duplicated(keep=False, subset=["Delivery Date", "Hour Ending"]), :]

,Delivery Date,Hour Ending,Settlement Point,Repeated Hour Flag,Settlement Point Price
0,01/01/2022,00:00,SUOMI,Y,4280.0
365,01/01/2022,00:00,SUOMI,Y,10470.0
24,01/02/2022,00:00,SUOMI,Y,11150.0
366,01/02/2022,00:00,SUOMI,Y,7970.0
48,01/03/2022,00:00,SUOMI,Y,4670.0
...,...,...,...,...,...
848,04/29/2023,00:00,SUOMI,Y,7690.0
2855,04/30/2023,00:00,SUOMI,Y,3390.0
849,04/30/2023,00:00,SUOMI,Y,3440.0
0,06/01/2023,00:00,SUOMI,Y,590.0


In [50]:
oomis.loc[oomis.duplicated(keep="first", subset=["Delivery Date", "Hour Ending"]), :]

,Delivery Date,Hour Ending,Settlement Point,Repeated Hour Flag,Settlement Point Price
365,01/01/2022,00:00,SUOMI,Y,10470.0
366,01/02/2022,00:00,SUOMI,Y,7970.0
367,01/03/2022,00:00,SUOMI,Y,10550.0
72,01/04/2022,00:00,SUOMI,Y,4860.0
369,01/05/2022,00:00,SUOMI,Y,15610.0
...,...,...,...,...,...
2783,04/27/2023,00:00,SUOMI,Y,5550.0
2807,04/28/2023,00:00,SUOMI,Y,11260.0
848,04/29/2023,00:00,SUOMI,Y,7690.0
849,04/30/2023,00:00,SUOMI,Y,3440.0


In [51]:
oomis.loc[oomis["Repeated Hour Flag"] == "Y", :]

,Delivery Date,Hour Ending,Settlement Point,Repeated Hour Flag,Settlement Point Price
0,01/01/2022,00:00,SUOMI,Y,4280.0
365,01/01/2022,00:00,SUOMI,Y,10470.0
24,01/02/2022,00:00,SUOMI,Y,11150.0
366,01/02/2022,00:00,SUOMI,Y,7970.0
48,01/03/2022,00:00,SUOMI,Y,4670.0
...,...,...,...,...,...
848,04/29/2023,00:00,SUOMI,Y,7690.0
2855,04/30/2023,00:00,SUOMI,Y,3390.0
849,04/30/2023,00:00,SUOMI,Y,3440.0
0,06/01/2023,00:00,SUOMI,Y,590.0


In [53]:
oomis_1 = oomis.drop_duplicates(subset=["Delivery Date", "Hour Ending"], keep="first")
get_data_by_year_and_month(oomis_1, 2023, "Jan")

,Delivery Date,Hour Ending,Settlement Point,Repeated Hour Flag,Settlement Point Price
730,01/01/2023,00:00,SUOMI,Y,2460.0
1,01/01/2023,01:00,SUOMI,N,810.0
2,01/01/2023,02:00,SUOMI,N,740.0
3,01/01/2023,03:00,SUOMI,N,600.0
4,01/01/2023,04:00,SUOMI,N,600.0
...,...,...,...,...,...
739,01/31/2023,19:00,SUOMI,N,12760.0
740,01/31/2023,20:00,SUOMI,N,12760.0
741,01/31/2023,21:00,SUOMI,N,11900.0
742,01/31/2023,22:00,SUOMI,N,11660.0


In [54]:
# write to excel using loop
#  get_data_by_year_and_month(oomis, 2021, "Jan") to FI/SPP/2021/rpt.00013060.0000000000000000.DAMLZHBSPP_2021.xlsx in sheet name Jan
#  get_data_by_year_and_month(oomis, 2021, "Feb") to FI/SPP/2021/rpt.00013060.0000000000000000.DAMLZHBSPP_2021.xlsx in sheet name Feb
# ...
#  get_data_by_year_and_month(oomis, 2021, "Dec") to FI/SPP/2021/rpt.00013060.0000000000000000.DAMLZHBSPP_2021.xlsx in sheet name Dec
# get_data_by_year_and_month(oomis, 2022, "Jan") to FI/SPP/2022/rpt.00013060.0000000000000000.DAMASMCPC_2022.xlsx in sheet name Jan
# ...
# get_data_by_year_and_month(oomis, 2024, "Mar") to FI/SPP/2024/rpt.00013060.0000000000000000.DAMASMCPC_2024.xlsx in sheet name Dec
for year in range(2021, 2025):
    os.makedirs(f"FI/SPP/{year}", exist_ok=True)
    for month in range(1, 13):
        month_name = pd.to_datetime(month, format="%m").month_name()[:3]
        data = get_data_by_year_and_month(oomis.drop_duplicates(subset=["Delivery Date", "Hour Ending"], keep="first"),
                                          year, month_name)
        data["Repeated Hour Flag"] = "N"
        # if not os.path.exists(f"FI/SPP/{year}/rpt.00013060.0000000000000000.DAMLZHBSPP_{year}.xlsx"):
        with pd.ExcelWriter(f"FI/SPP/{year}/rpt.00013060.0000000000000000.DAMLZHBSPP_{year}.xlsx", mode="a", if_sheet_exists="replace") as writer:
            data.to_excel(writer, sheet_name=month_name, index=False)
        # else:
        #     # use a writer in append mode
        #     with pd.ExcelWriter(f"FI/SPP/{year}/rpt.00013060.0000000000000000.DAMLZHBSPP_{year}.xlsx", mode="a") as writer:
        #         data.to_excel(f"FI/SPP/{year}/rpt.00013060.0000000000000000.DAMLZHBSPP_{year}.xlsx", sheet_name=month_name, index=False)
        

ValueError: Append mode is not supported with xlsxwriter!

In [ ]:
get_data_by_year_and_month(oomis, 2024, "Mar").loc[get_data_by_year_and_month(oomis, 2024, "Mar")["Delivery Date"].astype(str)=="03/31/2024"]

,Delivery Date,Hour Ending,Settlement Point,Repeated Hour Flag,Settlement Point Price
2160,03/31/2024,00:00,SUOMI,N,5810.0
2161,03/31/2024,01:00,SUOMI,N,5810.0
2162,03/31/2024,02:00,SUOMI,N,5800.0
2163,03/31/2024,04:00,SUOMI,N,5810.0
2164,03/31/2024,05:00,SUOMI,N,6020.0
2165,03/31/2024,06:00,SUOMI,N,6320.0
2166,03/31/2024,07:00,SUOMI,N,6120.0
2167,03/31/2024,08:00,SUOMI,N,6660.0
2168,03/31/2024,09:00,SUOMI,N,6790.0
2169,03/31/2024,10:00,SUOMI,N,6700.0


In [ ]:
get_data_by_year_and_month(oomis, 2024, "Mar").groupby("Delivery Date")["Hour Ending"].count().sort_values()#.shape 

Delivery Date
03/31/2024    23
03/29/2024    24
03/28/2024    24
03/27/2024    24
03/26/2024    24
03/25/2024    24
03/24/2024    24
03/23/2024    24
03/22/2024    24
03/21/2024    24
03/20/2024    24
03/19/2024    24
03/18/2024    24
03/17/2024    24
03/30/2024    24
03/01/2024    24
03/14/2024    24
03/13/2024    24
03/12/2024    24
03/11/2024    24
03/10/2024    24
03/09/2024    24
03/08/2024    24
03/07/2024    24
03/06/2024    24
03/05/2024    24
03/04/2024    24
03/03/2024    24
03/02/2024    24
03/15/2024    24
03/16/2024    24
Name: Hour Ending, dtype: int64

In [ ]:
get_data_by_year_and_month(oomis, 2024, "Mar")["Delivery Date"].unique()

array(['03/01/2024', '03/02/2024', '03/03/2024', '03/04/2024',
       '03/05/2024', '03/06/2024', '03/07/2024', '03/08/2024',
       '03/09/2024', '03/10/2024', '03/11/2024', '03/12/2024',
       '03/13/2024', '03/14/2024', '03/15/2024', '03/16/2024',
       '03/17/2024', '03/18/2024', '03/19/2024', '03/20/2024',
       '03/21/2024', '03/22/2024', '03/23/2024', '03/24/2024',
       '03/25/2024', '03/26/2024', '03/27/2024', '03/28/2024',
       '03/29/2024', '03/30/2024', '03/31/2024'], dtype=object)

In [ ]:
get_data_by_year_and_month(oomis, 2023, "Jan")#.groupby("Delivery Date")["Hour Ending"].count().unique()

,Delivery Date,Hour Ending,Settlement Point,Repeated Hour Flag,Settlement Point Price
730,01/01/2023,00:00,SUOMI,Y,2460.0
0,01/01/2023,00:00,SUOMI,Y,1120.0
1,01/01/2023,01:00,SUOMI,N,810.0
2,01/01/2023,02:00,SUOMI,N,740.0
3,01/01/2023,03:00,SUOMI,N,600.0
...,...,...,...,...,...
739,01/31/2023,19:00,SUOMI,N,12760.0
740,01/31/2023,20:00,SUOMI,N,12760.0
741,01/31/2023,21:00,SUOMI,N,11900.0
742,01/31/2023,22:00,SUOMI,N,11660.0


In [ ]:
get_data_by_year_and_month(oomis, 2023, "Oct").duplicated(subset=["Delivery Date", "Hour Ending"]).sum()

0

In [ ]:
os.listdir("FI/CCP/2021")

['rpt.00013091.0000000000000000.20210101080006.DAMASMCPC_2021.csv']

In [ ]:
ccp = pd.read_csv('FI/CCP/2021/rpt.00013091.0000000000000000.20210101080006.DAMASMCPC_2021.csv')

In [ ]:
ccp

,Delivery Date,Hour Ending,Repeated Hour Flag,REGDN,REGUP,RRS,NSPIN,CROSSBORDER
0,2021-01-01,00:00,N,18.50000,24.94999,82.79999,25,0.00
1,2021-01-01,01:00,N,24.35000,24.35000,84.09999,0,0.00
2,2021-01-01,02:00,N,23.98000,23.98000,84.09999,0,0.00
3,2021-01-01,03:00,N,23.71999,23.71999,84.09999,0,0.00
4,2021-01-01,04:00,N,23.73000,23.73000,85.79999,0,0.00
...,...,...,...,...,...,...,...,...
8755,2021-12-31,19:00,N,73.98000,73.98000,74.40000,25,89617.28
8756,2021-12-31,20:00,N,33.96000,44.95000,77.40000,25,58968.64
8757,2021-12-31,21:00,N,51.03000,51.03000,76.09999,25,76256.00
8758,2021-12-31,22:00,N,35.14999,46.54999,72.09999,25,42746.00


In [ ]:
# increase Delivery Date by 1 year
ccp["Delivery Date"] = pd.to_datetime(ccp["Delivery Date"], format="%m/%d/%Y") + pd.DateOffset(years=1)

ValueError: time data "2021-01-01" doesn't match format "%m/%d/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
ccp.to_csv("FI/CCP/2021/rpt.00013091.0000000000000000.20210101080006.DAMASMCPC_2021.csv", index=False)

In [ ]:
ccp

,Delivery Date,Hour Ending,Repeated Hour Flag,REGDN,REGUP,RRS,NSPIN,CROSSBORDER
0,2021-01-01,00:00,N,18.50000,24.94999,82.79999,25,0.00
1,2021-01-01,01:00,N,24.35000,24.35000,84.09999,0,0.00
2,2021-01-01,02:00,N,23.98000,23.98000,84.09999,0,0.00
3,2021-01-01,03:00,N,23.71999,23.71999,84.09999,0,0.00
4,2021-01-01,04:00,N,23.73000,23.73000,85.79999,0,0.00
...,...,...,...,...,...,...,...,...
8755,2021-12-31,19:00,N,73.98000,73.98000,74.40000,25,89617.28
8756,2021-12-31,20:00,N,33.96000,44.95000,77.40000,25,58968.64
8757,2021-12-31,21:00,N,51.03000,51.03000,76.09999,25,76256.00
8758,2021-12-31,22:00,N,35.14999,46.54999,72.09999,25,42746.00


In [ ]:
ccp["Delivery Date"] = pd.to_datetime(ccp["Delivery Date"], format="%m/%d/%Y") + pd.DateOffset(years=1)
ccp.to_csv("FI/CCP/2022/rpt.00013091.0000000000000000.20210101080006.DAMASMCPC_2022.csv", index=False)

ccp["Delivery Date"] = pd.to_datetime(ccp["Delivery Date"], format="%m/%d/%Y") + pd.DateOffset(years=1)
ccp.to_csv("FI/CCP/2023/rpt.00013091.0000000000000000.20210101080006.DAMASMCPC_2023.csv", index=False)

ccp["Delivery Date"] = pd.to_datetime(ccp["Delivery Date"], format="%m/%d/%Y") + pd.DateOffset(years=1) 
ccp = ccp.loc[ccp["Delivery Date"] < "2024-04-01"]
ccp.to_csv("FI/CCP/2024/rpt.00013091.0000000000000000.20210101080006.DAMASMCPC_2024.csv", index=False)

ValueError: time data "2021-01-01" doesn't match format "%m/%d/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.